In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr
import os
import pandas as pd
import scipy.stats
import sys
sys.path.insert(1, '/glade/u/home/lettier/analysis/')
import master_utils as myf
xr.set_options(keep_attrs=True)
from xgcm import Grid

We need to interpolate CESM1 output onto pressure levels

In [ ]:
def regrid_xgcm(ds, var):
    # Calculate the pressure at every time, lev, lat, lon
    p = (ds['hyam']*ds['P0'] + ds['hybm']*ds['PS'])/100
    # Assign it as a variable in the dataset
    ds = ds.assign({'p': np.log(p)})
    
    # Create an xgcm Grid object with a Z coordinate given by 'lev'
    grid = Grid(ds, coords={'Z': {'center': 'lev'}}, periodic=False)
    
    # Give the array of pressures to interpolate to
    p_target = np.array([10., 20., 30., 50., 70., 100., 150., 200., 250.,
                         300., 400., 500., 600., 650., 700., 750., 800., 850.,
                         900., 950., 1000.])
    
    # Use the transform method to interpolate to constant pressure given the target pressures above
    # The target_data parameter tells it what variable to use to base the transformation on. 
    # In our case, we're using the model pressure at every point calculated above
    varout = grid.transform(ds[var], 'Z', np.log(p_target), target_data=ds.p)
    
    # Rename the new dimension and assign the coordinate as the target pressures above
    varout = varout.rename({'p': 'plev'})
    varout = varout.assign_coords({'plev': p_target})
    
    return varout.squeeze()

In [ ]:
myvariables = ['Z3','T','PS','hybm','hyam','P0']
mytime = pd.date_range(start="1979-01-01",end="2018-12-31", freq='M')
edir = '/glade/scratch/wriggles/archive/'
ldir = '/glade/scratch/lettier/ed_archive/'
mydir = '/glade/work/lettier/NUDGE/'

In [ ]:
def wrangle_nudge (nudge_name, myvariables):
    ds_a = xr.open_mfdataset(edir+nudge_name+'/atm/hist/*.h0.*')[myvariables]
    ds_b = xr.open_mfdataset(edir+nudge_name+'_21C/atm/hist/*.h0.*')[myvariables]
    ds = xr.concat([ds_a,ds_b],dim='time')
    ds['time'] = mytime
    ds['names'] = nudge_name
    ds = ds.set_coords('names')
    
    return ds

In [ ]:
def wrangle_lens (e, myvariables):
    ledir = '/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/'

    ens = str(e)
    if e<10:
        ens = '0'+str(e)
    tmp = []
    print(ens)
    for var in ['Z3','PS','T']:
        myfiles = sorted([ledir+var+'/'+f for f in os.listdir(ledir+var) if ('B20TRC5CNBDRD' in f or 'BRCP85C5CNBDRD' in f) and '0'+ens+'.cam.h0.'+var in f ])
        myfiles = [f for f in myfiles if '.192001-199912.nc' not in f and '208101-210012.nc' not in f]
        ds = xr.open_mfdataset(myfiles).sel(time=slice('1979-02','2019-01'))
        ds['time'] = mytime
        tmp.append(ds)
        
    ds = xr.merge(tmp)
    ds['names'] = 'LENS'+ens
    ds = ds.set_coords('names')
    
    return ds

In [ ]:
def get_timeseries (inds):
    name = str(inds.names.values)
    
    
    listds = []
    for seas in ['DJF','MAM','JJA','SON']:
        seasds = inds.where(inds['time.season'] == seas).groupby('time.year').mean(dim='time')
    
        ds_a = regrid_xgcm(seasds, 'Z3').to_dataset(name='Z3')
        ds_b = regrid_xgcm(seasds, 'T').to_dataset(name='T')
        ds = xr.merge([ds_a,ds_b])[['T','Z3']]
        ds = ds.sel(plev=[200.,500.])
        print(ds)
   
        
        slope, intercept, r_value, p_value, std_err = myf.linregress(ds.year,ds.load(),dim='year')
    
        for var in ['T','Z3']:
            slope[var].attrs['units'] = ds[var].attrs['units']+'/yr'
            p_value[var] = 100.*p_value[var]
            p_value[var].attrs['units'] = '%'
            slope = slope.rename({var:var+'_trend'})
            p_value = p_value.rename({var:var+'_p_value'})

        seasds = xr.merge([slope, p_value])
        listds.append(seasds)
    ds = xr.concat(listds,dim='season')
                   
    
    ds['names'] = name
    ds = ds.set_coords('names')
    ds.attrs['desc'] = 'processed by Lettie Roach, June 2021'
    print(ds)
    ds.to_netcdf(mydir+'processed/spatial_mean_trend/atm/'+str(name)+'.atm_Z3T_plev_climtrend.1979-2018.nc')
    
    return ds    
    

In [ ]:
for run in ['anom_nudge_era_60_arclo']:
    print(run)
    get_timeseries(wrangle_nudge(run, myvariables))

In [ ]:
for e in range(1,36,1):
    ds = (wrangle_lens(e, myvariables))
    get_timeseries(ds)

Grab relevant stuff for ERA-Interim

In [ ]:
ds_z = xr.open_dataset('/glade/work/lettier/ERAI/mon/remap_cesmgrid/ei.moda.an.pl.regn128sc.1979-2018_z_remapcesmagrid.nc')
ds_t = xr.open_dataset('/glade/work/lettier/ERAI/mon/remap_cesmgrid/ei.moda.an.pl.regn128sc.1979-2018_t_remapcesmagrid.nc')
ds = xr.merge([ds_z,ds_t])
ds = ds.rename({'isobaricInhPa':'plev','z':'Z3','t':'T'})
ds['Z3'] = ds.Z3/9.81
ds = ds.sel(time=slice('1979','2018'))
name = 'ERAI'
ds.Z3.attrs = {}
ds.Z3.attrs['units'] = 'm'

ds = ds.sel(plev=[200.,500.])
ds['time'] = mytime
print(ds)

listds = []
for seas in ['DJF','MAM','JJA','SON']:

    seasds = ds.where(ds['time.season'] == seas).groupby('time.year').mean(dim='time')

    slope, intercept, r_value, p_value, std_err = myf.linregress(seasds.year,seasds.load(),dim='year')

    for var in seasds:
        slope[var].attrs['units'] =ds[var].attrs['units']+'/yr'
        p_value[var] = 100.*p_value[var]
        p_value[var].attrs['units'] = '%'
        slope = slope.rename({var:var+'_trend'})
        p_value = p_value.rename({var:var+'_p_value'})

    seasds = xr.merge([slope, p_value])
    seasds['season'] = seas
    seasds = seasds.set_coords('season')
    listds.append(seasds)
ds = xr.concat(listds,dim='season')


ds['names'] = name
ds = ds.set_coords('names')
ds.attrs['desc'] = 'processed by Lettie Roach, June 2021'
print(ds)
ds.to_netcdf(mydir+'processed/spatial_mean_trend/atm/'+str(name)+'.atm_Z3T_plev_climtrend.1979-2018.nc')
